In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 2
setup_one_gpu(GPU)

Picking GPU 2


In [2]:
import sys
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out import create_dir

from geo_tool import Point_Cloud

from tf_lab.point_clouds.ae_templates import mlp_architecture_ala_iclr_18, default_train_params
from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.convenience import reconstruct_pclouds

from tf_lab.data_sets.shape_net import snc_category_to_synth_id
from tf_lab.iclr.helper import load_multiple_version_of_pcs, find_best_validation_epoch_from_train_stats

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
top_data_dir = '/orions4-zfs/projects/optas/DATA/'
experiment_tag =  'mlp_with_split_3pc_usampled_bnorm_on_encoder_only'

n_pc_points = 2048
random_seed = 42

loss = 'chamfer'
class_name = raw_input('Give me the class type: ').lower()
syn_id = snc_category_to_synth_id()[class_name]

Give me the class type: chair


In [5]:
in_data = load_multiple_version_of_pcs('uniform_all', syn_id, n_classes=1)

Loading test data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/test.txt
679 pclouds were loaded. They belong in 1 shape-classes.
679 pclouds were loaded. They belong in 1 shape-classes.
679 pclouds were loaded. They belong in 1 shape-classes.
Loading train data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/train.txt
5761 pclouds were loaded. They belong in 1 shape-classes.
5761 pclouds were loaded. They belong in 1 shape-classes.
5761 pclouds were loaded. They belong in 1 shape-classes.
Loading val data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/val.txt
338 pclouds were loaded. They belong in 1 shape-classes.
338 pclouds were loaded. They belong in 1 shape-classes.
338 pclouds were loaded. They belong in 1 shape-classes.


In [6]:
bneck_list = [16]

for bneck in bneck_list:
    experiment_id = '_'.join(['ae', class_name, experiment_tag, str(n_pc_points), 'pts', str(bneck), 'bneck', loss])
    train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/', experiment_id)
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    
    val_error, best_epoch = find_best_validation_epoch_from_train_stats(osp.join(train_dir, 'train_stats.txt'))
    
    print bneck, val_error, best_epoch
    
    if best_epoch % conf.saver_step != 0: # Model was not saved at that epoch.
        best_epoch += best_epoch % conf.saver_step
    
    print best_epoch
    
    conf.encoder_args['verbose'] = False
    conf.decoder_args['verbose'] = False
    
    reset_tf_graph()
    ae = PointNetAutoEncoder(conf.experiment_name, conf)    
    ae.restore_model(conf.train_dir, best_epoch, verbose=True)

    _, l_val = reconstruct_pclouds(ae, in_data['val'].point_clouds, batch_size=100)    
    _, l_test = reconstruct_pclouds(ae, in_data['test'].point_clouds, batch_size=100)
    _, l_train = reconstruct_pclouds(ae, in_data['train'].point_clouds, batch_size=100)
    print l_val, l_test, l_train

16 0.00081458 205
210
Model restored in epoch 210.
0.000832818342691 0.000815450786325 0.000538709997306
